# Описание:
Собрать и подготовить свой собственный датасет, можно в соответствии с темой будущего проекта.
Аугментировать данные техниками из занятия.

## Данные:
Свой датасет

## Задачи:
1) Взять предобученную модель. 
    * Cтандартную предобученную модель из библиотек PyTorch `torchvision.models`
    * Примеры: ResNet18, VGG16, EfficientNet.
2) Применить несколько стратегий аугментации данных 
    * Определение стратегий аугментации
    * Пример Torchvision: `transforms.RandomHorizontalFlip`, `transforms.ColorJitter`.
    * Пример Albumentations: `HorizontalFlip`, `RandomBrightnessContrast`, `Cutout`, `ShiftScaleRotate`.
    * Пример сложных: <i>MixUp</i>, <i>CutMix</i>, <i>Mosaic</i>, <i>Weather</i>, <i>Style Transfer</i>.
3) Собрать результаты успешности стратегий в сводную табличку.
    * Базовая аугментация (поворот + отражение).
    * Цветовая аугментация (яркость, контраст, HSV).
    * Геометрическая аугментация (масштабирование, искажение).
    * Сложная аугментация (Cutout + MixUp).

Большим плюсом будет применение Pytorch Lightning для упорядочивания вашего проекта, но это не обязательное требование.


## Критерий оценки:
1. Есть сводная табличка в конце со сравнением стратегий.
2. У каждой стратегии есть краткое понятное описание, что именно сделано.
3. Есть визуализация собранного датасета.



In [25]:
import torch
import torchvision
#from torchvision.models.detection import FasterRCNN
#from torchvision.models.detection import AnchorGenerator
import albumentations as A
from torchvision import transforms as T
import cv2
import matplotlib.pyplot as plt
import os
import json
from torch.utils.data import Dataset

In [26]:

class CocoDetectionDataset(Dataset):
    # Конструктор класса
    def __init__(self, data_dir, annotation_file, transforms=None):
        self.data_dir = data_dir
        self.transforms = transforms
        with open(annotation_file, 'r') as file:
            self.coco = json.load(file)
        
        self.image_ids = [img['id'] for img in self.coco['images']]
        self.imgs = {img['id']: img for img in self.coco['images']}
        self.anns = {}
        for ann in self.coco['annotations']:
            if ann['image_id'] not in self.anns:
                self.anns[ann['image_id']] = []
            self.anns[ann['image_id']].append(ann)

    def __len__(self):
        return len(self.image_ids)

    def __getitem__(self, idx):
        image_id = self.image_ids[idx]
        img_info = self.imgs[image_id]
        img_path = os.path.join(self.data_dir, img_info['file_name'])
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        anns = self.anns.get(image_id, [])
        boxes = []
        labels = []
        for ann in anns:
            x, y, w, h = ann['bbox']
            boxes.append([x, y, x + w, y + h])  # COCO format to [x_min, y_min, x_max, y_max]
            labels.append(ann['category_id'])
        
        if self.transforms:
            transformed = self.transforms(image=image, bboxes=boxes, class_labels=labels)
            image = transformed['image']
            boxes = transformed['bboxes']
            labels = transformed['class_labels']
        
        target = {
            'boxes': torch.as_tensor(boxes, dtype=torch.float32),
            'labels': torch.as_tensor(labels, dtype=torch.int64),
            'image_id': torch.tensor([image_id])
        }
        
        return image, target

In [27]:
# Базовая аугментация
basic_transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.Rotate(limit=30, p=0.5),
    A.ToTensorV2()
], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['class_labels']))

# Цветовая аугментация
color_transform = A.Compose([
    A.RandomBrightnessContrast(p=0.5),
    A.HueSaturationValue(p=0.5),
    A.ToTensorV2()
], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['class_labels']))

# Геометрическая аугментация
geo_transform = A.Compose([
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=0, p=0.5),
    A.Perspective(p=0.3),
    A.ToTensorV2()
], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['class_labels']))

# Сложная аугментация (Cutout + MixUp)
# Для MixUp потребуется кастомная реализация, см. примечания ниже.

In [ ]:
# Создание датасетов и DataLoader
train_dataset = CocoDetectionDataset(
    root_dir='Lesson 9/Homework/DataForModel/images/Train',
    annotation_file='Lesson 9/Homework/DataForModel/annotations/instances_Train.json',
    transforms=basic_transform  # Меняйте трансформы здесь
)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=2, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))

TypeError: CocoDetectionDataset.__init__() got an unexpected keyword argument 'root_dir'

In [16]:

def get_model(num_classes):
    backbone = torchvision.models.resnet50(pretrained=True)
    backbone = backbone.backbone
    anchor_generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512),),
                                       aspect_ratios=((0.5, 1.0, 2.0),))
    model = FasterRCNN(backbone,
                       num_classes=num_classes,
                       rpn_anchor_generator=anchor_generator)
    return model

num_classes = 1  # Замените на ваше количество классов
model = get_model(num_classes)

c:\Users\khari\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\khari\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


AttributeError: 'ResNet' object has no attribute 'backbone'

In [1]:
import pytorch_lightning as pl
from torch.utils.data import DataLoader

class DetectionModel(pl.LightningModule):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def training_step(self, batch, batch_idx):
        images, targets = batch
        loss_dict = self.model(images, targets)
        loss = sum(loss for loss in loss_dict.values())
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-4)

# Создание датасетов и DataLoader
train_dataset = CocoDetectionDataset(
    root_dir='dataset/train/images',
    annotation_file='dataset/train/annotations/instances_train.json',
    transforms=basic_transform  # Меняйте трансформы здесь
)
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))

# Обучение
model = DetectionModel(get_model(num_classes))
trainer = pl.Trainer(max_epochs=10, gpus=1)
trainer.fit(model, train_loader)

NameError: name 'CocoDetectionDataset' is not defined